In [1]:
import os
from constants import huggingfaceHub_api_token

os.environ['HUGGINGFACEHUB_API_TOKEN']=huggingfaceHub_api_token

In [2]:
from langchain import HuggingFaceHub
llm=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

d:\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
d:\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
output=llm.predict("What is the capital of Russia")
output

d:\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'moscow'

<!-- Prompt Templates -->

In [4]:
from langchain.prompts import PromptTemplate

prompt=PromptTemplate(input_variables=['country'],
                      template="Tell me the capital of {country}")

prompt.format(country="India")

'Tell me the capital of India'

In [5]:
from langchain.chains import LLMChain

chain=LLMChain(llm=llm,prompt=prompt)
chain.run("India")

d:\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
d:\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'calcutta'

In [6]:
prompt=PromptTemplate(input_variables=['country'],
                      template="The capital of India is New Delhi. Tell me the capital of {country}")

prompt.format(country="India")

'The capital of India is New Delhi. Tell me the capital of India'

In [7]:
chain=LLMChain(llm=llm,prompt=prompt)
chain.run("India")

'New Delhi'

<!-- Combining Multiple chains Simple sequential chain -->

In [8]:
capital_prompt=PromptTemplate(input_variables=['country'],
                      template="Tell me the capital of {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_prompt)

In [9]:
place_prompt=PromptTemplate(input_variables=['capital'],
                      template="Tell me one famous place located in {capital}")

place_chain=LLMChain(llm=llm,prompt=place_prompt)

In [10]:
from langchain.chains import SimpleSequentialChain

chain=SimpleSequentialChain(chains=[capital_chain,place_chain])
chain.run("India")

'narayan temple'

<!-- sequential chain -->

In [11]:
capital_prompt=PromptTemplate(input_variables=['country'],
                      template="Tell me the capital of {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_prompt,output_key="capital")

In [12]:
place_prompt=PromptTemplate(input_variables=['capital'],
                      template="Tell me one famous place located in {capital}")

place_chain=LLMChain(llm=llm,prompt=place_prompt,output_key="place")

In [13]:
from langchain.chains import SequentialChain

chain=SequentialChain(chains=[capital_chain,place_chain],
                      input_variables=['country'],
                      output_variables=['capital','place'])

chain({'country':"India"})

d:\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'India', 'capital': 'calcutta', 'place': 'narayan temple'}

<!-- Chat model using OpenAI -->

In [14]:
from constants import openai_key
from langchain.chat_models import ChatOllama
# from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

# os.environ["OPENAI_API_KEY"]=openai_key

In [15]:
# chatllm=ChatOllama(temperature=0.6,model='llama3',ollama_server_url = "http://localhost:11434")
chatllm=ChatOllama(model='llama3')
# chatllm=ChatOpenAI(temperature=0.6)
chatllm

ChatOllama(model='llama3')

In [16]:
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines in AI")
])

d:\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='Here are some AI-themed comedy punchlines:\n\n1. "I\'m not arguing, I\'m just explaining why my algorithm is right... again."\n2. "Why did the AI go to therapy? It had a glitch in its self-awareness!"\n3. "What do you call an AI that\'s having a bad day? A bug-gerous malfunction!"\n4. "AI: Because humans are too lazy to write their own code... or think critically."\n5. "I told my AI assistant to make me a joke, but it just kept iterating until it was boring."\n6. "Why did the AI go on a diet? To optimize its computational overhead!"\n7. "What do you call an AI that\'s always tired? A slow-learned processor!"\n8. "AI: The only technology where \'beta testing\' is just a euphemism for \'it still doesn\'t work\'."\n9. "Why did the AI go on a date? To see if it was compatible... with its own programming."\n10. "What do you call an AI that\'s always complaining? A nag-bot-4000!"\n\nHope these make you LOL!', response_metadata={'model': 'llama3', 'created_at': '2024-05-06T

<!--Prompt Template+LLM+Ouput Parsers  -->

In [17]:
# from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [18]:
class Commasepratedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [19]:
template="You are a helpful assistant. When a user gives any input, you should generate 5 words synonyms in a comma separated manner."
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [20]:
chain=chatprompt|chatllm|Commasepratedoutput()

In [21]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' wise', ' brilliant', ' discerning']